데이터 로딩:

torchvision.datasets를 사용하여 MNIST 데이터셋을 다운로드하고, 전처리(transform)를 적용한 후 DataLoader로 배치 단위 데이터를 불러옵니다.

모델 구축:

간단한 신경망(예: 두 개의 Fully Connected Layer로 구성된 모델)을 정의합니다.

학습:

손실 함수(CrossEntropyLoss)와 옵티마이저(Adam)를 설정하고, 학습 루프를 통해 모델을 학습합니다.

평가:

테스트 데이터를 사용하여 모델의 정확도를 측정합니다.

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader as D
from torchvision import datasets, transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))]
)

train_data=datasets.MNIST(root="./data",train=True,download=True,transform=transform)
test_data=datasets.MNIST(root='./data',train=False,download=True,transform=transform)

train_loader=D(train_data,batch_size=64,shuffle=True)
test_loader=D(test_data,batch_size=1000, shuffle=False)

class Simple(nn.Module):
    def __init__(self):
        super(Simple,self).__init__()
        self.fc1=nn.Linear(28*28,128)  #(784, 128)28x28 이미지를 784 차원으로 펼쳐서 128 차원으로 변환  
        self.fc2=nn.Linear(128,10)  #128 차원에서 10 차원으로 변환 (각 클래스에 해당하는 출력)

    def forward(self, x):
        x=x.view(-1,28*28)  #-1은 PyTorch에서 해당 차원을 자동으로 계산하라는 의미(batch_size, 784)
        x=torch.relu(self.fc1(x))
        x=self.fc2(x)
        return x


model=Simple()
print("모델 구조:\n", model)

모델 구조:
 Simple(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [19]:
import torch.optim as optim

criter=nn.CrossEntropyLoss()
#optim=optim.Adam(model.parameters(),lr=0.001)
optim=optim.Adam(model.parameters(),lr=0.001)

num_epoch=10

for epoch in range(num_epoch):
    model.train() # 모델을 학습 모드로 전환
    total=0  # 에폭 동안의 누적 손실을 저장할 변수
    for batch,(data,target) in enumerate(train_loader):
        optim.zero_grad()  # 이전 배치에서 계산된 gradient를 초기화
        out=model(data)  # forward pass: 입력 데이터를 모델에 통과시켜 예측값(out)을 계산
        loss=criter(out,target)  # 손실 함수로 예측값과 실제 타깃(target) 간의 오차 계산
        loss.backward()  # backward pass: 손실에 대해 미분(gradient)을 계산
        optim.step(   # 옵티마이저가 계산된 gradient를 사용하여 파라미터를 업데이트
        total+=loss.item()  # 현재 배치의 손실 값을 누적

    avg_loss=total/len(train_loader)  # 에폭 동안의 평균 손실 계산
    print(f"Epoch {epoch+1}/{num_epoch}, Training Loss: {avg_loss:.4f}")


model.eval()
correct=0
totalE=0
with torch.no_grad():
    for data,target in test_loader:
        out=model(data)
        pred=out.argmax(dim=1, keepdim=True)
        correct+=pred.eq(target.view_as(pred)).sum().item()
        totalE+=data.size(0)

acc=correct/totalE*100
print(f"Test Accuracy: {acc:.2f}%")


Epoch 1/10, Training Loss: 0.0081
Epoch 2/10, Training Loss: 0.0054
Epoch 3/10, Training Loss: 0.0070
Epoch 4/10, Training Loss: 0.0075
Epoch 5/10, Training Loss: 0.0093
Epoch 6/10, Training Loss: 0.0046
Epoch 7/10, Training Loss: 0.0056
Epoch 8/10, Training Loss: 0.0081
Epoch 9/10, Training Loss: 0.0061
Epoch 10/10, Training Loss: 0.0029
Test Accuracy: 97.90%


nn.CrossEntropyLoss()는 분류 문제에서 자주 사용하는 손실 함수

이 손실 함수는 모델의 출력(예측값)과 실제 정답(클래스 라벨) 간의 차이를 계산합니다.

옵티마이저(Optimizer):

optim.Adam(model.parameters(), lr=0.001)는 Adam 옵티마이저를 사용하여 모델의 파라미터를 업데이트합니다.

model.parameters()는 학습 가능한 모든 파라미터를 반환하며,
학습률(learning rate)을 0.001로 설정합니다.


-----------------------------------------------------------------------------------
에폭(Epoch):

num_epoch = 10으로 10번 전체 데이터를 학습합니다.

모델 학습 모드:

model.train()를 호출하여, 드롭아웃 같은 학습 전용 기능을 활성화합니다.

배치 순회:

for batch, (data, target) in enumerate(train_loader):

train_loader는 데이터셋을 배치 단위로 반환합니다.

enumerate를 사용하여 각 배치의 인덱스(batch)와 데이터를 동시에 얻습니다.

data와 target은 각각 입력 데이터와 정답(라벨)을 나타냅니다.

Gradient 초기화:

optim.zero_grad()를 호출하여, 이전 배치에서 남은 gradient를 초기화합니다.

Forward Pass:

out = model(data)를 통해, 입력 데이터를 모델에 넣어 예측 결과를 얻습니다.

손실 계산:

loss = criter(out, target)를 통해, 예측 결과와 실제 값 간의 손실을 계산합니다.

Backward Pass:

loss.backward()는 손실 함수의 결과에 대해 역전파를 수행하여 각 파라미터의 gradient를 계산합니다.

파라미터 업데이트:

optim.step()은 계산된 gradient를 바탕으로 파라미터를 업데이트합니다.

손실 누적 및 평균:

각 배치의 손실 값을 누적한 후, 전체 배치 수로 나눠 평균 손실을 계산하여 출력합니다.

----------------------------------------------------------------------------------------------

모델 평가 모드:

model.eval()을 호출하여, 평가 모드로 전환합니다.

평가 모드에서는 dropout과 배치 정규화 같은 레이어들이 학습 모드와 다르게 동작합니다.

Gradient 비활성화:

with torch.no_grad(): 블록 내부에서는 gradient 계산을 하지 않으므로, 메모리와 계산 속도가 효율적입니다.

예측 및 정확도 계산:

테스트 데이터셋을 배치 단위로 불러와 model(data)로 예측을 수행합니다.

out.argmax(dim=1, keepdim=True)를 통해, 각 샘플마다 가장 높은 점수를 받은 클래스 인덱스를 얻습니다.

{
out.argmax(dim=1, keepdim=True)에서 dim=1은 argmax 연산을 수행할 차원을 지정하는 것입니다.

예를 들어, 만약 out의 shape가 (batch_size, num_classes)라면,

dim=0은 각 열(클래스에 해당하는 각 요소들) 중에서 최대값을 찾으려는 의미이고,
dim=1은 각 행(각 샘플마다)의 요소들 중에서 최대값의 인덱스를 찾겠다는 의미입니다.

즉, 보통 분류 문제에서는 모델의 출력이 (batch_size, num_classes) 형태이므로,
dim=1을 지정하여 각 샘플별로 가장 높은 확률(또는 로짓)을 가진 클래스의 인덱스를 구합니다.

keepdim=True는 원래의 차원을 유지하여 결과를 반환하는 옵션으로,
예를 들어 출력이 (batch_size, 1)의 형태로 유지됩니다.

이렇게 해서 각 샘플에 대해 예측된 클래스 인덱스를 얻을 수 있습니다.

}

pred.eq(target.view_as(pred))를 통해, 예측된 클래스와 실제 클래스가 일치하는지를 확인합니다.

{
eq는 파이토치에서 텐서 객체에 내장된 메서드입니다.

**pred.eq(other)**는 pred의 각 요소와 other의 각 요소를 비교하여, 같은 위치에서 값이 일치하면 True, 그렇지 않으면 False인 불리언 텐서를 반환합니다.

이 메서드는 예약어(reserved keyword)가 아니라, 파이토치(Torch)에서 제공하는 표준 함수입니다.

또한,

**target.view_as(pred)**는 target 텐서를 pred와 동일한 형태로 변환하는 메서드로, 두 텐서의 shape이 일치하도록 만듭니다.

이렇게 하면 pred.eq(target.view_as(pred))는 두 텐서의 각 요소를 비교할 수 있게 됩니다.
결론적으로, eq는 예약어가 아니라 파이토치에서 제공하는 텐서 메서드입니다.

view_as는 필수적으로 사용해야 하는 예약어는 아니에요.

역할:

target.view_as(pred)는 target 텐서의 shape를 pred와 동일하게 바꿔줍니다.

필요성:

만약 target의 shape가 이미 pred와 같다면, 굳이 사용할 필요는 없습니다.

하지만, 종종 target의 shape가 다를 수 있으므로, element-wise 비교를 위해 두 텐서의 shape가 일치하는지 확인하는 안전 장치로 사용됩니다.

결론적으로, 두 텐서의 shape를 확실하게 맞추고자 할 때 view_as를 사용하는 것이 좋지만, 상황에 따라 생략할 수 있습니다.

}

이를 통해 맞춘 샘플의 개수를 correct에 누적하고, 전체 샘플 수는 totalE에 누적합니다.

최종 정확도:

acc = correct / totalE * 100로 정확도를 계산하고, 퍼센트로 변환하여 출력합니다.
